In [1]:
import sys
sys.path
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import time
import os
import re
import csv
DEBUG = 1
from datetime import datetime, timedelta
import dateparser
import calendar

In [2]:
def convertnumber(number):
    if 'k' in number:
        n=float(number.replace('k', '').replace(' ', ''))*1000
    elif 'm' in number:
        n=float(number.replace('m', '').replace(' ', ''))*1000000
    else:
        n=number
    return int(n)  

In [3]:
def scrolldown(browser):        
    src_updated = browser.page_source
    src = ""
    while  src != src_updated:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        src = src_updated
        src_updated = browser.page_source  

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
def connectchrome():

    # instantiate a chrome options object so you can set the size and headless preference
    options = Options()
    options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    driver = webdriver.Chrome(options=options)
    driver.execute_script('window.scrollTo(0, 0);')
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
    # and put it in the current directory
    chromedriver = "/home/youcef/Documents/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver     
    browser = webdriver.Chrome(options=options) 
    browser.get('https://www.quora.com/')
    time.sleep(2)
    return browser

In [5]:
def getTopic():
    file_question_topics = open((os.path.join('/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping'+'/topic2.txt')), mode='r', encoding='utf-8')
    lines=file_question_topics.readlines()
    line=lines[0]
    topic=line.strip()
    return str(topic.strip('\n'))
    

In [6]:
def crawlURL():
    # read topics form a file
    print(sys.path[0])
    file_question_topics = open((os.path.join('/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping'+'/topic2.txt')), mode='r', encoding='utf-8')
    cc=0
    driver=connectchrome()
    lines=file_question_topics.readlines()
    parsing_loop=True
    print(len(lines))
    while parsing_loop:
        line=lines[cc]
        cc+=1
        topic=line.strip()
        if (topic.startswith("#")):
            continue

       # give the url to scrape

        try:
            print('###################')
            print('starting new topic: ' + str(topic))
            url = "https://www.quora.com/topic/" + topic.strip('\n') + "/all_questions"
            print(str(url))
            driver.get(url)
        except Exception as e0:
            print('exception e0')
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e0).__name__, e0)
            continue     


        # define pause time for browser
        SCROLL_PAUSE_TIME = 2

        # get browser source
        html_source = driver.page_source
        question_count_soup = BeautifulSoup(html_source, 'html.parser')

        #  get total number of questions
        question_count_str = question_count_soup.find('a', attrs={'class': 'TopicQuestionsStatsRow'})

        if str(question_count_str)=='None':
            print('topic does not exist in Quora')
            continue

        question_count = question_count_str.contents[0].text 
        if 'k' in str(question_count):
            question_count = str(question_count).replace('k','')
            question_count = int(float(question_count)*1000)
            #print(type(question_count))
        print('number of questions for this topic : '+ str(question_count))

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        question_set = set()
        # infinite while loop, break it when you reach the end of the page or not able to scroll further.
        if int(question_count)>10:
            start_time_sd = time.time()
            max_time=  int(question_count)*0.25
            if int(question_count)> 8000:
                max_time=1800
            while True:
                i = 0
                # try to scroll 5 times in case of slow connection
                while True:
                    # Scroll down to one page length
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load page
                    time.sleep(SCROLL_PAUSE_TIME)

                    # get page height in pixels
                    new_height = driver.execute_script("return document.body.scrollHeight")

                    # break this loop when you are able to scroll futher
                    if new_height != last_height:
                        break
                    if i==1:
                        SCROLL_PAUSE_TIME = 4   
                    if i==2:
                        if int(question_count)>500 and int(question_count)<2000:
                            SCROLL_PAUSE_TIME = 7
                        elif int(question_count)>2000:
                            SCROLL_PAUSE_TIME = 15    
                        elif int(question_count)<500:
                            break
                    if i==3:
                        break
                    i += 1
            SCROLL_PAUSE_TIME = 2    
            # not able to scroll further, break the infinite loop
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            total_time=time.time() - start_time_sd
            if total_time>max_time:
                print('max time exceeded')
                break



        print('getting ressources ...')
        # get html page source
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        # question_link is the class for questions
        question_link = soup.find_all('a', attrs={'class': 'question_link'}, href=True)

        # add questions to a set for uniqueness
        for ques in question_link:
            question_set.add(ques)


        # write content of set to a file called question_urls.txt
        questions_directory = 'topics/'
        os.makedirs('topics/', exist_ok=True)
        file_name = questions_directory + '/' + topic.strip('\n') + '_question_urls.txt'
        file_question_urls = open(file_name, mode='w', encoding='utf-8')
        writer = csv.writer(file_question_urls)
        for ques in question_set:
            link_url = "http://www.quora.com" + ques.attrs['href']
            #print(link_url)
            writer.writerows([[link_url]])

        #sleeping each 300 requests
        if cc % 300== 0:
            print('quitting chrome')
            driver.quit()
            time.sleep(6)
            driver=connectchrome()
        #loop_limit can be adjusted based on the number of requests we want to send 
        loop_limit=2
        if cc>loop_limit:
            parsing_loop=False

    # finish time
    end_time = datetime.datetime.now()
    print(end_time-start_time)

In [7]:
def extractQuestionTitle(driver):
    try:
        question_title = driver.find_element_by_xpath('.//span[contains(@class, "rendered_qtext")]').text
    except:
        question_title = "error"
    return question_title

In [8]:
def extractAnswerCount(driver):
    try:
        answer_count = driver.find_element_by_xpath("//div[@class = 'answer_count']")
        text_count = answer_count.text.split()
        count = int((text_count[0].split('+'))[0])
    except:
        count = "error"
    return count

In [9]:
def extractTopics(driver):
    try:
        topic_list = []
        topics = driver.find_elements_by_xpath(".//span[@class = 'TopicName TopicNameSpan']")
        for topic in topics:
            topic_list.append(topic.text)
    except:
        topic_list = []
    return topic_list

In [10]:
def scrollDownPage(driver):
    current_scroll_position, new_height= 0, 1
    speed = 10
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [11]:
def expandAllAnswers(driver):
    hidden_list = driver.find_elements_by_xpath(".//div[@class='ui_qtext_truncated ui_qtext_truncated_regular']/div[@class='ui_qtext_truncated_text']")
    #print('Number of hidden answer:',len(hidden_list))
    for hidden in hidden_list:
        hidden = hidden.find_element_by_xpath(".//a[@class='ui_qtext_more_link']")
        webdriver.ActionChains(driver).move_to_element(hidden).click(hidden).perform()

In [12]:
def extractRespond(driver):
    check_ans_count = 0
    answers_list = {}
    name_list = []
    upvote_list = []
    ans_list = []
    date_list = []
    view_list = []
    error_list = []
    question_title = ''
    check_ans_count = 0
    try: 
        respond_list = driver.find_elements_by_xpath(".//div[contains(@class, 'AnswerBase') and contains(@class, 'AnswerBase') and contains(@class, 'Toggle') and contains(@class, 'Answer')]")
        for respond in respond_list:
            # Name
            ans = {}
            try:
                user_name = respond.find_element_by_xpath('.//a[@class = "user"]')
                name_list.append(user_name.text)
                ans['user_name'] = user_name.text
            except Exception:
                user_name = respond.find_element_by_xpath('.//span[contains(@class, "anon_user")]')
                name_list.append(user_name.text)
                ans['user_name'] = user_name.text
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                name_list.append('')
                ans['user_name'] = 'error'
                print(error_list[-1])

            # Date
            try:
                date = respond.find_element_by_xpath(".//a[@class = 'answer_permalink']")
                date_split = date.text.split('Answered ')
                question_date = date_split[1]
                date_list.append(question_date)
                ans['date'] = question_date
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                date_list.append('')
                ans['date'] = 'error'
                print(error_list[-1])

            # Features that show after expand
            try:
                expanded_answer = respond.find_element_by_xpath(".//div[contains(@class,'ExpandedAnswer')]")
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                print(error_list[-1])
                continue

            # Answer
            try:
                answer = expanded_answer.find_element_by_xpath(".//div[@class='ui_qtext_expanded']/span[@class='ui_qtext_rendered_qtext']")
                ans_list.append(answer.text)
                ans['answer'] = answer.text
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                ans_list.append('')
                ans['answer'] = 'error'
                print(error_list[-1])

            # Answer upvote
            try:
                upvote = expanded_answer.find_element_by_xpath(".//a[@class='VoterListModalLink']")
                res = [int(i) for i in upvote.text.split() if i.isdigit()]
                upvote_list.append(res[0])
                ans['upvote'] = res[0]
            except Exception:
                upvote_list.append(0)
                ans['upvote'] = 0
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                upvote_list.append(0)
                ans['upvote'] = 'error'
                print(error_list[-1])

            # Answer view
            try:
                answer_footer = expanded_answer.find_element_by_xpath(".//div[contains(@class, 'AnswerFooter')]")
                view_text = (answer_footer.text.split())[0]
                if 'k' in view_text:
                    view = float((view_text.split('k'))[0]) * 1000
                elif 'm' in view_text:
                    view = float((view_text.split('m'))[0]) * 1000000
                else:
                    view = int(view_text)
                view_list.append(view)
                ans['view'] = view
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                view_list.append(0)
                ans['view'] = 'error'
                print(error_list[-1])

            print()    
            check_ans_count += 1
            print(type(ans))
            if check_ans_count > len(answers_list):
                answers_list[str(len(answers_list))] = ans
        print("Respond collected:", check_ans_count)

        return answers_list
    
    except Exception as e:
        error_list.append([e, question_title, check_ans_count])
        print(error_list[-1])
        return check_ans_count, name_list, upvote_list, ans_list, date_list, view_list, error_list
    
    

In [13]:
def scrollDownPageFaster(driver):
    current_scroll_position, new_height= 0, 1
    speed = 5
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [14]:
def getQuestionAddedBy(driver):
    try:
        last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
        user_name = last_item.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except:
        try:
            last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
            user_name = last_item.find_element_by_xpath(".//a[@class = 'user']").text
        except:
            return
    return user_name

In [15]:
def extractCreateTime(driver):
    try:
        last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
        question_data = last_item.find_element_by_xpath(".//p[@class = 'log_action_bar']")
        question_data_split = question_data.text.split('· ')
        question_date = question_data_split[1]
    except:
        question_date = ''
    return question_date

In [16]:
def extractNoOfFollower(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_follower = int((stats[0].text.split(' '))[0])
    except:
        no_of_follower = 'error'
    return no_of_follower

In [17]:
def extractNoOfView(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_view = (stats[1].text.split(' '))[0]
    except:
        no_of_view = 'error'
    return no_of_view

In [18]:
def extractQuestion(driver, url):
    # Extract Question
    data = {}
    scrollDownPage(driver)  
    expandAllAnswers(driver)
    data['question_title'] = extractQuestionTitle(driver)
    data['answer_count'] = extractAnswerCount(driver)
    data['topic_list'] = extractTopics(driver)
    data['respond'] = extractRespond(driver)
    driver.get(url+'/log')
    scrollDownPageFaster(driver)
    data['question_author'] = getQuestionAddedBy(driver)
    data['create_time'] = extractCreateTime(driver)
    data['no_of_follower'] = extractNoOfFollower(driver)
    data['no_of_view'] = extractNoOfView(driver)
    return data

In [19]:
def crawlQuestionData(file):
    start_time = time.time()
    #if (DEBUG): #print ("In crawlQuestionData...")
    
    # Open question url file
    file_question_urls = open(file, mode = 'r')
    topic = str(getTopic())
    topic = topic + "_answers.txt"

    #file_users = open(os.path.join(sys.path[0]+"users.txt", mode = 'a'))
    current_lines = file_question_urls.readlines()
    i=-1
    browser= connectchrome()
    k=0
    loop=True
    #for current_line in current_lines:
    dictdata = {}
    while  loop :
        try:
            current_line= current_lines[k]
        except:
            break
        print('processing question number  : '+ str(k))
        k+=1
        if '/unanswered/' in str(current_line):
            print('answer is unanswered')
            continue     
        print ("*************************************************************************")
        if (DEBUG): print(current_line)
        question_id = current_line
        browser.get(current_line)
        data = extractQuestion(browser, current_line)
        print(type(data))
        print(data)
        dictdata[str(len(dictdata))] = data
        print(type(dictdata))
        
        j = json.dumps(dictdata)
        # we save all retrieved answers into answers.txt file
        with open(topic, 'w') as file_answers:
            file_answers.write(j)
            file_answers.close()
    
        print ("*************************************************************************")
        # In our usecase 74256 is the loop limit value
        if k>=20000 :
           loop= False
    print("--- %s seconds ---" % (time.time() - start_time))        
    browser.quit()

In [20]:
def main():
    print(os.getcwd())
    #crawlURL()
    # We merged all Questions urls crawled by 1-Questions_URLs_crawler.py into one file (quora_urls.txt)
    crawlQuestionData((os.path.join(os.getcwd()+"/topics/" + getTopic() + "_question_urls.txt")))
    return 0

if __name__ == "__main__": main()    

/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping
processing question number  : 0
*************************************************************************
http://www.quora.com/My-family-against-for-my-love-marriage-because-of-astrology-can-someone-advise

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]

<class 'dict'>

<class 'dict'>

<class 'dict'>
Respond coll

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]

<class 'dict'>

<class 'dict'>

<class 'dict'>

<class 'dict'>
Respond collected: 4
<class 'dict'>
{'create_time': 'Apr 10, 2020', 'topic_list': ['Break Ups', 'Letting Go', 'Moving On', 'Breaking Up', 'Love', 'Relationship Advice', 'Dating Advice', 'Dating and Relationships', 'Life and Living', 'Life Advice'], 'respond': {'3': {'date': 'Apr 10, 2020', 'user_name': 'Aashi Sontakke', 'upvote': 0, 'view': 33, 'answer': "See it's about acceptance. It can take one day or six years. It depends on you and how you act on it. Write advantages and disadvantages after the bre


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'create_time': 'Apr 13, 2020', 'topic_list': ['Pretending', 'Husbands and Wives', 'Wives', 'Husbands', 'Marriage Advice', 'Marriage', 'Love', 'Relationship Advice', 'Dating and Relationships', 'Human Behavior'], 'respond': {'0': {'date': 'Apr 15, 2020', 'user_name': 'Raj Sekhar', 'upvote': 0, 'view': 25, 'answer': 'This is no big deal.\nthe couple start life, keep it going , achieve some success. Relationship is established. Bonding based on interaction is locked in.\nif one goes deviant, the trending in behaviour changes. Partner will ignore a couple of times. Not for long. Pretension is fought on. Leads to tension. Negative enviournment.\nif handled badly biginning of the end. For the couple.\njust take it head on . discuss. Settle. Move on. If not too you will move on. Opposite direction. That is all. All up to you.'}}, 'no_of_follower': 1, 'question_author': 'Jay Prakash Bhanja', 'question_title': 'How can a husband know if his w

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'create_time': 'Apr 16, 2020', 'topic_list': [], 'respond': {'0': {'date': 'Apr 16, 2020', 'user_name': 'Jason Taylor', 'upvote': 0, 'view': 10, 'answer': 'An Evangelical is a vague description of a cluster of Christian denominations. A romantic is someone who is inclined toward literature and art from the romantic era, and/or enjoys the glamorization of erotic love and emotion in general commonly found in such works. It is possible to be an Evangelical and be cold-blooded enough to put Admiral Spruance to shame, and it is also possible to be an Evangelical and prefer Mark Twain to Sir Walter Scot. It might make that person a bad fit (because the liturgy of Evangelicals, is sometimes rather, well, romantic and intellectual study of theology as a discipline is downplayed) but it does not prevent him being an Evangelical.'}}, 'no_of_follower': 1, 'question_author': 'Kathy R. England', 'question_title': 'What is the difference between a

...

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]

<class 'dict'>
Respond collected: 1
<class 'dict'>
{'create_time': 'Apr 14, 2020', 'topic_list': ['Familial Love', 'Mothers', 'Understanding Love', 'Family Relationships and Dynamics', 'Parents', 'Children', 'Family', 'Parenting', 'Emotions', 'Love', 'Personal Question'], 'respond': {'0': {'date': 'Apr 14, 2020', 'user_name': 'Geeta Shah', 'upvote': 1, 'view': 35, 'answer': "Whatever people may tell you children are nurtured slightly differently by parents on a subconscious level. Mothers tend to favour their 'weaker' child who is often the youngest one. The one who appears to be able to be more independent will get less attention from their mum especially. The ones who're always needy will get a lot more attention. Its probably just nature rather than anything sinister."}},


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'create_time': 'Apr 11, 2020', 'topic_list': ['Lesbians', 'Homosexuality', 'Girlfriends', 'LGBTQ', 'Love Life Advice', 'Love', 'Relationship Advice', 'Dating Advice', 'Dating and Relationships', 'Interpersonal Interaction'], 'respond': {'0': {'date': 'Apr 11, 2020', 'user_name': 'Kena Schumacher', 'upvote': 0, 'view': 55, 'answer': 'How about plan a romantic video date. Try googling tips for long distance couples and use those as guides. Or to really up the romance, recreate the scene from Love Actually where the guy is holding up handwritten signs to express his feelings for the girl. You could recreate the scene outside of the window or door :)'}}, 'no_of_follower': 3, 'question_author': 'Nichole Plath', 'question_title': '(Lesbians) I want to tell my girlfriend of 6 months I love her, but quarantine is keeping me from seeing her in person for a while. Should I wait or say it over text?', 'no_of_view': '1,480', 'answer_count': 1}
<

Respond collected: 0
<class 'dict'>
{'create_time': '', 'topic_list': [], 'respond': {}, 'no_of_follower': 'error', 'question_author': None, 'question_title': 'error', 'no_of_view': 'error', 'answer_count': 'error'}
<class 'dict'>
*************************************************************************
processing question number  : 1850
answer is unanswered
processing question number  : 1851
*************************************************************************
http://www.quora.comhttps://www.quora.com/q/yagwjubbdkhevkap/How-to-ask-of-me-and-another-person-are-dating-without-seeming-desperate-or-needy

Respond collected: 0
<class 'dict'>
{'create_time': '', 'topic_list': [], 'respond': {}, 'no_of_follower': 'error', 'question_author': None, 'question_title': 'error', 'no_of_view': 'error', 'answer_count': 'error'}
<class 'dict'>
*************************************************************************
processing question number  : 1852
**********************************************

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 2]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 2]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 2]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 2]

<class 'dict'>
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'Expan

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
Respond collected: 0
<class 'dict'>
{'create_time': 'Apr 11, 2020', 'topic_list': ['Letting Go', 'Using Psychology in Relationships', 'Dating and Relationship Advice', 'Love', 'Relationship Advice', 'Dating Advice', 'Dating and Relationships', 'Life Advice'], 'respond': {}, 'no_of_follower': 1, 'question_author': 'Katjah Omega', 'question_title': 'How do I know if I should just let a relationship go even if I’ve wanted it for the longest time?', 'no_of_view': '1,716', 'answer_count': 2}
<class 'dict'>
*****************************************************************